# Adding Back the Tides - CIOPS
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

#### unlike what we did with SSC in this file make all add back tides days seperate to reduce space requirement

In [2]:
startday = [dt.datetime(2015,11,22)+dt.timedelta(days=i) for i in range(int(1498))]
print(startday[-1])

2019-12-28 00:00:00


In [3]:
# for i in range(131,len(startday)-1):
#     print(i)

In [4]:
folders = [dt.datetime(2015,11,22)+dt.timedelta(days=7*(i+1)) for i in range(int(214))]
print(folders[-1])
folders = np.repeat(folders,7)

2019-12-29 00:00:00


In [5]:
# start = dt.datetime(2015,11,22) #start day of your run

#dates for each run
# numdays = 2 #15 for all except the last run (7) 
# date_list = [start + dt.timedelta(days=x) for x in range(numdays)]
# folderday = [start+ dt.timedelta(days=7),start+ dt.timedelta(days=7*2)]#,start+ dt.timedelta(days=7*3)]
# folderday = np.repeat(folderday,7)
# folderday = folderday[:-6]
date_list_daily = [startday[0],startday[1]]
folderday_daily = [folders[0], folders[1]]

# for whatever reason the naming it raelly really weird and the daily and the hourly files names for the SAME DAY aren't\
# on the same day.. must take hourly files a day ahead woooooooooooo
date_list_hourly = [startday[1],startday[2]]
folderday_hourly = [folders[1], folders[2]]

In [6]:
"{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[0], date_list_hourly[0], date_list_hourly[0])

'2015112900/BC12_1h_grid_U_2D_20151123_20151123.nc'

In [7]:
path = Path("/ocean/mdunphy/CIOPSW-BC12/")

drop_vars = (
    "sbu", "tauuo", "time_counter_bounds", "time_instant_bounds", "uos", "time_instant",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[i], date_list_hourly[i], date_list_hourly[i]))) for i in range(len(date_list_hourly))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
ut_h = mydata['ubar']

In [8]:
files

[[PosixPath('/ocean/mdunphy/CIOPSW-BC12/2015112900/BC12_1h_grid_U_2D_20151123_20151123.nc')],
 [PosixPath('/ocean/mdunphy/CIOPSW-BC12/2015112900/BC12_1h_grid_U_2D_20151124_20151124.nc')]]

In [9]:
ut_h

<xarray.DataArray 'ubar' (time_counter: 48, y: 1020, x: 714)>
dask.array<concatenate, shape=(48, 1020, 714), dtype=float32, chunksize=(24, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * time_counter  (time_counter) datetime64[ns] 2015-11-23T01:00:00 ... 2015-...
Dimensions without coordinates: y, x
Attributes:
    long_name:           ocean barotropic current along i-axis
    units:               m/s
    online_operation:    instant
    interval_operation:  60 s
    interval_write:      1 h
    cell_methods:        time: point (interval: 60 s)

In [10]:
ut_h[0,646,267].values

array(0.03388941)

In [11]:
drop_vars = (
    "sbv", "tauvo", "time_counter_bounds", "time_instant_bounds", "vos", "time_instant",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_V_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[i], date_list_hourly[i], date_list_hourly[i]))) for i in range(len(date_list_hourly))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
vt_h = mydata['vbar']

In [12]:
# 2D hourly outputs are alraedy in bartropic form so you dont need to do any conversion!

In [13]:
# Now get the required data from the daily files
drop_vars = (
    "depthu_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_U_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_daily[i], date_list_daily[i], date_list_daily[i]))) for i in range(len(date_list_daily))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['uo']

drop_vars = (
    "depthv_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_V_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_daily[i], date_list_daily[i], date_list_daily[i]))) for i in range(len(date_list_daily))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vo']

In [14]:
u_d

<xarray.DataArray 'uo' (time_counter: 2, depthu: 75, y: 1020, x: 714)>
dask.array<concatenate, shape=(2, 75, 1020, 714), dtype=float32, chunksize=(1, 75, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 2015-11-23 2015-11-24
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_x_velocity
    long_name:           Sea Water X Velocity
    units:               m/s
    online_operation:    instant
    interval_operation:  1 d
    interval_write:      1 d
    cell_methods:        time: point

In [15]:
u_d[0,:,646,267].values

array([0.04478236, 0.0435583 , 0.04354728, 0.04353595, 0.04352422,
       0.04351205, 0.04349937, 0.04348617, 0.04347137, 0.04309732,
       0.04215222, 0.04051778, 0.03733189, 0.02910914, 0.01988747,
       0.01769056, 0.01928514, 0.02016401, 0.02118212, 0.01872806,
       0.02191998, 0.01443028, 0.00311109, 0.01905654, 0.07829779,
       0.0486123 , 0.04513838, 0.00476805, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [16]:
# we WILL need to do some conversions here, so get e3t from the mesh_mask file
mydata = xr.open_dataset("/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc")
e3t = mydata['e3t_0']
e3t

<xarray.DataArray 'e3t_0' (t: 1, z: 75, y: 1020, x: 714)>
[54621000 values with dtype=float64]
Dimensions without coordinates: t, z, y, x

In [17]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'z': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'z': 'depthv'})

In [18]:
#calcuate bartropic component
ut_d = (u_d*e3u[0,:,:,:]).sum(dim='depthu')/e3u[0,:,:,:].sum(dim='depthu')

In [22]:
ut_d[0,646,267].values

array(0.00076539)

In [19]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [20]:
uc_d[0,:,646,267].values

array([ 0.04401698,  0.04279291,  0.04278189,  0.04277056,  0.04275884,
        0.04274666,  0.04273398,  0.04272079,  0.04270598,  0.04233193,
        0.04138684,  0.0397524 ,  0.03656651,  0.02834375,  0.01912208,
        0.01692517,  0.01851975,  0.01939863,  0.02041673,  0.01796267,
        0.02115459,  0.0136649 ,  0.00234571,  0.01829116,  0.07753241,
        0.04784691,  0.04437299,  0.00400266, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00076539,
       -0.00076539, -0.00076539, -0.00076539, -0.00076539, -0.00

In [21]:
offset = dt.timedelta(hours=1) # daily avg timestamp as next day time 0, offset by -23 hours to line up with hourly data
uc_h_interp = uc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")
#instead of taking 12hours off each side like when done with SSC, this method takes off 24hours on the end!
# , loffset=offset

In [22]:
uc_h_interp

<xarray.DataArray (time_counter: 25, depthu: 75, y: 1020, x: 714)>
dask.array<transpose, shape=(25, 75, 1020, 714), dtype=float64, chunksize=(25, 75, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 2015-11-23T01:00:00 ... 2015-...
Dimensions without coordinates: y, x

In [29]:
uc_h_interp[0,0,646,267].values

array(0.04304314)

In [23]:
u_new = ut_h  + uc_h_interp

In [24]:
u_new = u_new.isel(time_counter = np.arange(0,24,1))

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [34]:
u_new = u_new.rename('vozocrtx')

In [37]:
u_new

<xarray.DataArray 'vozocrtx' (time_counter: 24, y: 1020, x: 714, depthu: 75)>
dask.array<getitem, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2015-11-23T01:00:00 ... 2015-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [36]:
u_new[0,646,267,0].values

array(0.07693255)

In [39]:
encoding={
          "vozocrtx": {"zlib": True, "complevel": 4}
}

In [40]:
encoding

{'vozocrtx': {'zlib': True, 'complevel': 4}}

In [45]:
# encoding = {
#     "depthu": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#             "depthu": 75,
#         }
#     },
#     "vozocrtx": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#             "time_counter": 1, "y": 1020, "x": 714, "depthu": 75,
#         }
#     }
# }

encoding = {
    "depthu": {
        "zlib": True, "complevel": 4,
        "chunksizes": [75],
    },
    "vozocrtx": {
        "zlib": True, "complevel": 4,
        "chunksizes": {
        "time_counter": 1, "y": 1020, "x": 714, "depthu": 75,
        }
    }
}

In [46]:
encoding

{'depthu': {'zlib': True, 'complevel': 4, 'chunksizes': [75]},
 'vozocrtx': {'zlib': True,
  'complevel': 4,
  'chunksizes': {'time_counter': 1, 'y': 1020, 'x': 714, 'depthu': 75}}}

In [47]:
path = "/ocean/rbeutel/data/"
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_WithEncodingChunk.nc'.format(date_list_hourly[0],date_list_hourly[0]), encoding=encoding)
print('u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list_hourly[0],date_list_hourly[0]))

KeyError: 0

In [27]:
#calcuate bartropic component
vt_d = (v_d*e3v[0,:,:,:]).sum(dim='depthv')/e3v[0,:,:,:].sum(dim='depthv')

In [29]:
#subtract from v to get baroclinic component
vc_d = v_d-vt_d 

In [30]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")

In [31]:
v_new = vt_h  + vc_h_interp

In [32]:
v_new = v_new.isel(time_counter = np.arange(0,24,1))
v_new = v_new.rename('vomecrty')

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [33]:
v_new

<xarray.DataArray 'vomecrty' (time_counter: 24, y: 1020, x: 714, depthv: 75)>
dask.array<getitem, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2016-04-01T01:00:00 ... 2016-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
  * depthv        (depthv) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))